In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:

{"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }
]
}

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'about',
   'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ['I.m Joana, your bot assistant',
    "I'm Joana, an Artificial Intelligent bot"]},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Joana.',
    "I'm Joana!",
    'Just call me as Joana']},
  {'tag': 'help',
   'patterns': ['Could you help me?',
    'give me a hand please',
    'Can you help

In [3]:
with open('../Desktop/Python/DATASET/Untitled-1.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post' , maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu', input_shape= (28,28,1)))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='saprse_categorical_crossentropy',
             optimizer= 'adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [8]:
epochs = 550
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
2/2 [==============================] - 0s 6ms/step - loss: 2.0791 - accuracy: 0.2121
Epoch 2/550
2/2 [==============================] - 0s 2ms/step - loss: 2.0783 - accuracy: 0.2121
Epoch 3/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0779 - accuracy: 0.2121
Epoch 4/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0775 - accuracy: 0.2121
Epoch 5/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0773 - accuracy: 0.2121
Epoch 6/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0770 - accuracy: 0.2121
Epoch 7/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0768 - accuracy: 0.2121
Epoch 8/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0766 - accuracy: 0.2121
Epoch 9/550
2/2 [==============================] - 0s 2ms/step - loss: 2.0764 - accuracy: 0.2424
Epoch 10/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0762 - accuracy: 0.2727
Epoch 11/550
2/2 [===========

2/2 [==============================] - 0s 2ms/step - loss: 2.0299 - accuracy: 0.2121
Epoch 85/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0285 - accuracy: 0.2121
Epoch 86/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0272 - accuracy: 0.2121
Epoch 87/550
2/2 [==============================] - 0s 997us/step - loss: 2.0261 - accuracy: 0.2121
Epoch 88/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0251 - accuracy: 0.2121
Epoch 89/550
2/2 [==============================] - 0s 2ms/step - loss: 2.0241 - accuracy: 0.2121
Epoch 90/550
2/2 [==============================] - 0s 995us/step - loss: 2.0229 - accuracy: 0.2121
Epoch 91/550
2/2 [==============================] - 0s 2ms/step - loss: 2.0215 - accuracy: 0.2121
Epoch 92/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0201 - accuracy: 0.2121
Epoch 93/550
2/2 [==============================] - 0s 2ms/step - loss: 2.0189 - accuracy: 0.2121
Epoch 94/550
2/2 [===========

2/2 [==============================] - 0s 2ms/step - loss: 1.8608 - accuracy: 0.3333
Epoch 167/550
2/2 [==============================] - 0s 997us/step - loss: 1.8584 - accuracy: 0.3333
Epoch 168/550
2/2 [==============================] - 0s 2ms/step - loss: 1.8555 - accuracy: 0.3333
Epoch 169/550
2/2 [==============================] - 0s 2ms/step - loss: 1.8525 - accuracy: 0.3333
Epoch 170/550
2/2 [==============================] - 0s 2ms/step - loss: 1.8487 - accuracy: 0.3636
Epoch 171/550
2/2 [==============================] - 0s 2ms/step - loss: 1.8454 - accuracy: 0.3636
Epoch 172/550
2/2 [==============================] - 0s 2ms/step - loss: 1.8422 - accuracy: 0.3636
Epoch 173/550
2/2 [==============================] - 0s 2ms/step - loss: 1.8396 - accuracy: 0.3636
Epoch 174/550
2/2 [==============================] - 0s 2ms/step - loss: 1.8367 - accuracy: 0.3636
Epoch 175/550
2/2 [==============================] - 0s 2ms/step - loss: 1.8337 - accuracy: 0.3636
Epoch 176/550
2/2 [===

2/2 [==============================] - 0s 3ms/step - loss: 1.5661 - accuracy: 0.3636
Epoch 249/550
2/2 [==============================] - 0s 2ms/step - loss: 1.5636 - accuracy: 0.3636
Epoch 250/550
2/2 [==============================] - 0s 1ms/step - loss: 1.5607 - accuracy: 0.3636
Epoch 251/550
2/2 [==============================] - 0s 4ms/step - loss: 1.5582 - accuracy: 0.3636
Epoch 252/550
2/2 [==============================] - 0s 2ms/step - loss: 1.5556 - accuracy: 0.3636
Epoch 253/550
2/2 [==============================] - 0s 1ms/step - loss: 1.5515 - accuracy: 0.3636
Epoch 254/550
2/2 [==============================] - 0s 3ms/step - loss: 1.5471 - accuracy: 0.3636
Epoch 255/550
2/2 [==============================] - 0s 2ms/step - loss: 1.5436 - accuracy: 0.3636
Epoch 256/550
2/2 [==============================] - 0s 1ms/step - loss: 1.5408 - accuracy: 0.3636
Epoch 257/550
2/2 [==============================] - 0s 2ms/step - loss: 1.5383 - accuracy: 0.3636
Epoch 258/550
2/2 [=====

2/2 [==============================] - 0s 2ms/step - loss: 1.3822 - accuracy: 0.3636
Epoch 331/550
2/2 [==============================] - 0s 2ms/step - loss: 1.3796 - accuracy: 0.3636
Epoch 332/550
2/2 [==============================] - 0s 2ms/step - loss: 1.3766 - accuracy: 0.3636
Epoch 333/550
2/2 [==============================] - 0s 3ms/step - loss: 1.3737 - accuracy: 0.3636
Epoch 334/550
2/2 [==============================] - 0s 2ms/step - loss: 1.3714 - accuracy: 0.3636
Epoch 335/550
2/2 [==============================] - 0s 2ms/step - loss: 1.3694 - accuracy: 0.3636
Epoch 336/550
2/2 [==============================] - 0s 2ms/step - loss: 1.3682 - accuracy: 0.3636
Epoch 337/550
2/2 [==============================] - 0s 2ms/step - loss: 1.3653 - accuracy: 0.3636
Epoch 338/550
2/2 [==============================] - 0s 3ms/step - loss: 1.3619 - accuracy: 0.3939
Epoch 339/550
2/2 [==============================] - 0s 3ms/step - loss: 1.3589 - accuracy: 0.3939
Epoch 340/550
2/2 [=====

2/2 [==============================] - 0s 2ms/step - loss: 1.2083 - accuracy: 0.4545
Epoch 413/550
2/2 [==============================] - 0s 2ms/step - loss: 1.2051 - accuracy: 0.4545
Epoch 414/550
2/2 [==============================] - 0s 2ms/step - loss: 1.2015 - accuracy: 0.4545
Epoch 415/550
2/2 [==============================] - 0s 2ms/step - loss: 1.1983 - accuracy: 0.4545
Epoch 416/550
2/2 [==============================] - 0s 2ms/step - loss: 1.1956 - accuracy: 0.4545
Epoch 417/550
2/2 [==============================] - 0s 3ms/step - loss: 1.1931 - accuracy: 0.4545
Epoch 418/550
2/2 [==============================] - 0s 2ms/step - loss: 1.1906 - accuracy: 0.4545
Epoch 419/550
2/2 [==============================] - 0s 2ms/step - loss: 1.1884 - accuracy: 0.4545
Epoch 420/550
2/2 [==============================] - 0s 3ms/step - loss: 1.1864 - accuracy: 0.4545
Epoch 421/550
2/2 [==============================] - 0s 2ms/step - loss: 1.1844 - accuracy: 0.4545
Epoch 422/550
2/2 [=====

2/2 [==============================] - 0s 2ms/step - loss: 1.0291 - accuracy: 0.5455
Epoch 495/550
2/2 [==============================] - 0s 2ms/step - loss: 1.0289 - accuracy: 0.5455
Epoch 496/550
2/2 [==============================] - 0s 2ms/step - loss: 1.0269 - accuracy: 0.5455
Epoch 497/550
2/2 [==============================] - 0s 996us/step - loss: 1.0237 - accuracy: 0.5455
Epoch 498/550
2/2 [==============================] - 0s 2ms/step - loss: 1.0214 - accuracy: 0.5455
Epoch 499/550
2/2 [==============================] - 0s 2ms/step - loss: 1.0185 - accuracy: 0.5455
Epoch 500/550
2/2 [==============================] - 0s 2ms/step - loss: 1.0159 - accuracy: 0.5455
Epoch 501/550
2/2 [==============================] - 0s 2ms/step - loss: 1.0136 - accuracy: 0.5455
Epoch 502/550
2/2 [==============================] - 0s 2ms/step - loss: 1.0118 - accuracy: 0.5455
Epoch 503/550
2/2 [==============================] - 0s 2ms/step - loss: 1.0107 - accuracy: 0.5455
Epoch 504/550
2/2 [===

In [9]:
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [19]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("../Desktop/Python/DATASET/Untitled-1.json") as file:
    data = json.load(file)
    
    def chat():
        model = keras.models.load_model('chat_model')
        
        
        
        
        with open ('tokenizer.pickle', 'rb') as handle:
            tokenizer = pickle.load(enc)
                
                
        with open('label_encoder.pickle', 'rb') as enc:
            lbl_encoder = pickle.load(enc)
            
                
        max_len = 20
        
        
        while True:
            print(Fore.LIGHTBLACK_EX + "user: " + style.REST_ALL, end="")
            inp = input()
            if inp.lower() == "quit":
                break
                
                
                
            result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                                                              truncating='post' , 
                                                                              maxlen= max_len))
            
            tag = lbl_encoder.inverse_transform([np.argmax(result)])
            
           
            for i in data['intents']:
                if i['tag'] == tag:
                    print(Fore.GREEN + "ChatBot:" + style.RESET_ALL , np.random.choice(i['responses']))
                    
        print(Fore.YELLOW + "Start messaging with the bot (type quit in stop)!" + Style.RESET_ALL)
        
            
        chat()   

In [10]:
model.save("chat_model")

import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [11]:
def check():
    print('start talking with bot, Enter quit to exit')
    while True:
        string = input('SONI: ')
        if string == 'quit': break
        result = model.predict(pad_sequences(tokenizer.texts_to_sequences([string]),
                                             truncating='post', maxlen=max_len))
        category = lbl_encoder.inverse_transform([np.argmax(result)]) # labels[np.argmax(result)]
        for i in data['intents']:
            if i['tag']==category:
                print(np.random.choice(i['responses']))

In [ ]:
check()

start talking with bot, Enter quit to exit
SONI: hi
Tell me how can assist you
SONI: how are you
Yes Sure, How can I support you
SONI: tell me about you
Yes Sure, How can I support you
SONI: hello
Tell me your problem to assist you
